# 0) Импорт библиотек

In [1]:
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px

# 1) Загрузка данных

In [ ]:
data = pd.read_csv(r"sm_lab_data_prepared.csv")

data.head(3)

,device_type,event_id,user_id,user_is_authorized,location_id,create_timestamp,product_id,product_is_pick_up,product_is_pick_point,product_is_delivery,product_is_now,product_price,ret_discount,product_category,product_gender,product_age,brand_lvl,add_to_cart_flag,create_order_flag
0,desktop,cd4192b5-3709-46dd-a601-444dde3e762d,1,0,968,2024-06-02 10:46:21.989,68390,0.0,1.0,0.0,0.0,5280,950,инвентарь,унисекс,взрослые,1,0,0
1,desktop,dd8a2b1a-9268-4930-b414-cfbd9cda16d7,2,0,725,2024-06-08 20:03:52.614,46818,0.0,1.0,1.0,0.0,2599,1300,одежда,женский,взрослые,1,0,0
2,desktop,0416ecb1-e53a-474b-9cbc-ca3009dbaed6,3,0,968,2024-06-19 06:08:03.096,79467,0.0,1.0,1.0,0.0,5599,3471,одежда,мужской,дети,2,0,0


# 2) ABC - анализ 

Сводная таблица по категориям и уникальным товарам с расчетом полученной выручки и количества продаж товаров 

In [3]:
data_purchased = data[data["create_order_flag"] == 1]

In [4]:
abc_grouped = data_purchased.groupby(by = ["product_category", "product_id"])["product_price"].agg(["sum", "count"])
abc_grouped = abc_grouped.reset_index()

abc_grouped

,product_category,product_id,sum,count
0,инвентарь,5,129,1
1,инвентарь,12,3996,4
2,инвентарь,13,999,1
3,инвентарь,14,1998,2
4,инвентарь,15,999,1
...,...,...,...,...
21865,одежда,90122,6900,1
21866,одежда,90260,7320,1
21867,одежда,90420,698,2
21868,одежда,90550,799,1


In [5]:
revenue_part = []

revenue_sum = abc_grouped["sum"].sum()
for product_id in abc_grouped["product_id"]:
    revenue_part.append(abc_grouped["sum"][abc_grouped["product_id"] == product_id].sum() / revenue_sum * 100)

abc_grouped["revenue_part, %"] = revenue_part

In [6]:
quantity_part = []

quantity = abc_grouped["count"].sum()
for product_id in abc_grouped["product_id"]:
    quantity_part.append(abc_grouped["count"][abc_grouped["product_id"] == product_id].sum() / quantity * 100)

abc_grouped["quantity_part, %"] = quantity_part

In [7]:
abc_grouped.sort_values(by = ["revenue_part, %", "quantity_part, %"], ascending = False)

,product_category,product_id,sum,count,"revenue_part, %","quantity_part, %"
2492,инвентарь,45024,10037197,303,0.666579,0.086497
7419,обувь,54327,7636874,1246,0.507171,0.355692
4975,обувь,348,7518966,1314,0.499341,0.375104
2494,инвентарь,45026,7389708,242,0.490757,0.069083
2493,инвентарь,45025,6926938,262,0.460024,0.074792
...,...,...,...,...,...,...
9953,одежда,11388,79,1,0.000005,0.000285
421,инвентарь,8571,64,1,0.000004,0.000285
47,инвентарь,289,49,1,0.000003,0.000285
1676,инвентарь,30141,49,1,0.000003,0.000285


In [8]:
colors_list = px.colors.qualitative.Prism

In [31]:
fig = go.Figure()

fig.add_trace(go.Box(y = abc_grouped["revenue_part, %"], boxpoints='outliers', 
                     boxmean = True, name = "Процент от суммарной выручки", marker_color = colors_list[1]))
fig.add_trace(go.Box(y = abc_grouped["quantity_part, %"], boxpoints='outliers', 
                     boxmean=True, name = "Процент от суммарного товарооборота", marker_color = "rgb(228,26,28)"))

fig.update_layout(showlegend = False)
fig.update_layout(height = 700, width = 1200)
fig.update_layout(title = "Распределение значений полученных величин")
fig.update_layout(font_family = "Arial", font_size = 13, title_font_family = "Arial", font_color = "black")
fig.update_yaxes(title_text = "Процент")

fig.show()

In [17]:
print(f"В среднем выручка от реализации одного типа товара составляет {round(abc_grouped['revenue_part, %'].mean(), 5)}% от суммарного объема продаж. \nПо категориям: ")
for cat in abc_grouped["product_category"].unique():
    print(f'{cat} - {round(abc_grouped["revenue_part, %"][abc_grouped["product_category"] == cat].mean(), 5)}%')

В среднем выручка от реализации одного типа товара составляет 0.00457% от суммарного объема продаж. 
По категориям: 
инвентарь - 0.00465%
обувь - 0.01038%
одежда - 0.00243%


In [18]:
print(f"В среднем число реализованных единиц одного типа товара составляет {round(abc_grouped['quantity_part, %'].mean(), 5)}% от общего товарооборота. \nПо категориям: ")
for cat in abc_grouped["product_category"].unique():
    print(f'{cat} - {round(abc_grouped["quantity_part, %"][abc_grouped["product_category"] == cat].mean(), 5)}%')

В среднем число реализованных единиц одного типа товара составляет 0.00457% от общего товарооборота. 
По категориям: 
инвентарь - 0.00286%
обувь - 0.00775%
одежда - 0.00411%


In [19]:
abc_grouped["metric"] = abc_grouped["quantity_part, %"] * 0.3 + abc_grouped["revenue_part, %"] * 0.7
abc_grouped.sort_values(by = "metric", ascending = False)

,product_category,product_id,sum,count,"revenue_part, %","quantity_part, %",metric
2492,инвентарь,45024,10037197,303,0.666579,0.086497,0.492554
4975,обувь,348,7518966,1314,0.499341,0.375104,0.462070
7419,обувь,54327,7636874,1246,0.507171,0.355692,0.461728
2494,инвентарь,45026,7389708,242,0.490757,0.069083,0.364255
2493,инвентарь,45025,6926938,262,0.460024,0.074792,0.344455
...,...,...,...,...,...,...,...
5,инвентарь,28,79,1,0.000005,0.000285,0.000089
421,инвентарь,8571,64,1,0.000004,0.000285,0.000089
47,инвентарь,289,49,1,0.000003,0.000285,0.000088
1676,инвентарь,30141,49,1,0.000003,0.000285,0.000088


In [21]:
fig = go.Figure()

fig.add_trace(go.Box(y = abc_grouped["metric"], boxpoints='outliers', marker_color = colors_list[1], boxmean = True, name = ""))

fig.update_layout(showlegend = False)
fig.update_layout(height = 700, width = 1200)
fig.update_layout(title = "Распределение значений полученной метрики")
fig.update_layout(font_family = "Arial", font_size = 13, title_font_family = "Arial", font_color = "black")
fig.update_yaxes(title_text = "Значение")

fig.show()

In [22]:
def abc(x):
    if x > 0.0038:
        return "A"
    elif x < 0.0004:
        return "C"
    else:
        return "B"
    
abc_grouped["class"] = abc_grouped["metric"].apply(abc)

In [23]:
abc_grouped[["product_category", "product_id", "sum", "count", "metric", "class"]].sort_values(by = ["class", "metric"], ascending = [True, False]).head(10)

,product_category,product_id,sum,count,metric,class
2492,инвентарь,45024,10037197,303,0.492554,A
4975,обувь,348,7518966,1314,0.462070,A
7419,обувь,54327,7636874,1246,0.461728,A
2494,инвентарь,45026,7389708,242,0.364255,A
2493,инвентарь,45025,6926938,262,0.344455,A
5540,обувь,30451,4511040,1160,0.309050,A
864,инвентарь,20324,5565959,441,0.296515,A
7427,обувь,54335,4585562,578,0.262672,A
5541,обувь,30452,3558309,941,0.246005,A
4974,обувь,347,3976462,698,0.244633,A


In [24]:
abc_grouped[["product_category", "product_id", "sum", "count", "metric", "class"]].sort_values(by = ["class", "metric"], ascending = [True, False]).tail(10)

,product_category,product_id,sum,count,metric,class
9554,одежда,1694,99,1,0.000090,C
9630,одежда,3540,99,1,0.000090,C
5,инвентарь,28,79,1,0.000089,C
715,инвентарь,18276,79,1,0.000089,C
4285,инвентарь,78460,79,1,0.000089,C
9953,одежда,11388,79,1,0.000089,C
421,инвентарь,8571,64,1,0.000089,C
47,инвентарь,289,49,1,0.000088,C
1676,инвентарь,30141,49,1,0.000088,C
1679,инвентарь,30144,49,1,0.000088,C


In [25]:
fig = go.Figure()

fig.add_trace(go.Bar(x = abc_grouped["class"].value_counts().keys(), y = abc_grouped["class"].value_counts().values, marker_color = colors_list[1]))

fig.update_layout(height = 700, width = 1200)
fig.update_layout(title_text = "Количество товаров каждой группы")
fig.update_layout(font_family = "Arial", font_size = 13, title_font_family = "Arial", font_color = "black")
fig.update_xaxes(title_text = "Группа")
fig.update_yaxes(title_text = "Количество товаров")

fig.show()

In [30]:
fig = go.Figure()

revenue_per_class = abc_grouped.groupby(by = "class")["sum"].sum().reset_index()
revenue_per_class["%"] = revenue_per_class["sum"].apply(lambda x: x / revenue_per_class["sum"].sum() * 100).cumsum()

fig.add_trace(go.Bar(x = revenue_per_class["class"], y = revenue_per_class["sum"], marker_color = colors_list[1]))
fig.add_trace(go.Scatter(x = revenue_per_class["class"], y = revenue_per_class["%"], marker_color = "rgb(228,26,28)", yaxis= "y2"))

fig.update_layout(height = 700, width = 1200,
                  yaxis1 = dict(title = "Выручка, руб."),
                  yaxis2=dict(overlaying = "y", range = [0, 110],
                              side='right', rangemode='tozero', tickvals=[0, 20, 40, 60, 80, 100]))
fig.update_layout(title_text = "Выручка от продажи товаров каждой группы")
fig.update_layout(font_family = "Arial", font_size = 13, title_font_family = "Arial", font_color = "black",
                  plot_bgcolor="rgb(255, 255, 255)", showlegend = False)
fig.update_xaxes(title_text = "Группа")

fig.show()

Результаты анализа соответствуют классической методике:

Группа A: Товары, составляющие около 84% общей выручки

Группа B: Товары, составляющие следующие около 15% выручки 

Группа C: Товары, составляющие оставшиеся около 1% выручки 

# 84% выручки приносит продажа позиций группы A